# 入站认证 (Inbound Auth)

Agent Identity 让您可以验证用户和应用程序调用 AgentKit Platform 中的agents/tools，或者以AgentKit Gatway为目标的入站访问（Inbound Auth）。它还提供从agents到外部服务或AgentKit Gatway为目标的安全出站访问（OutBound Auth）。它与您现有的身份提供商（如 Agent Identity）集成，同时为独立行动或代表用户行动的agents（通过 OAuth）强制执行权限边界。

入站身份认证会验证试图调用Agent或工具的来访方的身份，无论这个Agent或者和工具是否跑在AgentKit Runtime还是其他环境中都可以使用。

默认情况下，AgentKit Runtime 使用 OAuth2，在配置 AgentKit Runtime 或 AgentKit Gateway 时，作为Token 的验证方（也就是流量的被访问方）需要配置以下内容：

- OAuth Discovery Server URL OAuth2 发现服务器 URL — 必须与 OpenID Connect 发现 URL 的模式 ^.+/\.well-known/openid-configuration$匹配的字符串

- Allowed clients 允许的客户端 — 允许的客户端标识符client_id 列表

如果您使用 AgentKit CLI，可以在使用 **config** 命令时为 AgentKit Runtime 指定授权类型（和 OAuth 发现服务器）。您也可以使用 CreateRuntime 操作和 Agent Identity 控制台。如果您正在创建网关，则使用 CreateGateway 操作或控制台。

在用户可以使用 Agent 之前，客户端应用程序必须让用户使用 OAuth 授权器进行身份验证。您的客户端接收一个承载令牌（Bearer token），然后在调用请求中将其传递给 Agent 。收到后， Agent 在允许访问之前使用OAuth2的授权服务器验证令牌。


## 概述

在本教程中，我们将修改您使用 [AgentKit Runtime 快速开始](https://volcengine.github.io/agentkit-sdk-python/content/4.runtime/1.runtime_quickstart.html) 提供的 Agent，并使用 Agent Identity 作为身份提供商为其配置入站认证。您将设置一个包含一个用户和一个应用客户端的 Agent Identity 用户池。您将学习如何使用 AgentKit Platform 托管现有 Agent ，并使用 Agent Identity 用户池进行入站认证。

### Tutorial 架构

<div style="text-align:center">
    <img src="images/inbound_auth_identity.png" width="90%"/>
</div>

### Tutorial 说明


| 信息                | 详情                                                                          |
|:--------------------|:---------------------------------------------------------------------------------
|  Agent 类型            | 单Agent                                                                           |
|  Agent 框架            | Volcengine Agent Development Kit(VeADK) Agents                                                                   |
| Tutorial 核心模块            | AgentKit Runtime, Agent Identity |
| 教程复杂度          | 简单                                                                             |
| 入站认证            | Agent Identity                                                                          |
| 使用的 SDK          | AgentKit Python SDK, VeADK, volcengine-python-sdk                                    |



### 教程关键特性

* 在 AgentKit Platform 上托管 Agent ，使用 Agent Identity 进行入站认证
* 使用火山方舟模型
* 使用 AgentKit Python SDK


## 教程

### 先决条件

要执行本教程，您需要：
* Python 3.10+
* Volcengine 凭据
* AgentKit Python SDK 
* VeADK
* [uv](https://docs.astral.sh/uv/#scripts)



In [ ]:
# 您可以通过 `--index` 参数指定镜像源以加快安装速度。
!uv pip install --force-reinstall -U -r requirements.txt -q

设置火山引擎 AccessKey 和 SecretKey 以及方舟 API Key 来使用 Agent Identity 接口以及进行 AgentKit Runtime 部署

In [ ]:
import os
from pathlib import Path
from dotenv import load_dotenv

# 检查 .env 文件是否存在
env_file = Path('.env')
env_template = Path('.env.template')

if not env_file.exists():
    if env_template.exists():
        raise FileNotFoundError(
            "未找到 .env 文件！\n"
            "请先复制模板文件：cp .env.template .env\n"
            "然后填写您的配置信息。"
        )
    else:
        raise FileNotFoundError(
            "未找到 .env 文件！请创建 .env 文件并设置以下环境变量：\n"
            "- VOLCENGINE_ACCESS_KEY\n"
            "- VOLCENGINE_SECRET_KEY\n"
            "- MODEL_API_KEY"
        )

load_dotenv()

VOLCENGINE_ACCESS_KEY = os.getenv("VOLCENGINE_ACCESS_KEY")
VOLCENGINE_SECRET_KEY = os.getenv("VOLCENGINE_SECRET_KEY")
MODEL_API_KEY = os.getenv("MODEL_API_KEY")

if not VOLCENGINE_ACCESS_KEY:
    raise ValueError("请设置 VOLCENGINE_ACCESS_KEY 环境变量")

if not VOLCENGINE_SECRET_KEY:
    raise ValueError("请设置 VOLCENGINE_SECRET_KEY 环境变量")

if not MODEL_API_KEY:
    raise ValueError("请设置 MODEL_API_KEY 环境变量")

### 设置 Agent Identity 进行身份验证

让我们配置一个包含应用客户端和一个测试用户的 Agent Identity 用户池，目的是用这个测试用户演示Agent如何验证应用程序中的来访用户的身份。 我们将使用 Agent Identity 提供 JWT 令牌来访问我们在AgentKit Runtime 上托管 Agent。

为此，我们将使用 `utils` 脚本中的 `setup_veidentity_user_pool` 支持函数。同时这个函数也会生成下面步骤配置OAuth Inbound Authorizer 会用到的两个参数：discovery_url 和allowedClients

注意：Agent Identity access_token 仅有效 10 小时。如果 access_token 过期，您可以使用 `reauthenticate_user` 方法生成另一个 access_token。

In [ ]:
import sys
import os
from agentkit.toolkit import sdk

# 获取当前 notebook 的目录
current_dir = os.path.dirname(
    os.path.abspath("__file__" if "__file__" in globals() else ".")
)
utils_dir = os.path.abspath(current_dir)

# 添加到 sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import (
    setup_veidentity_user_pool,
    reauthenticate_user,
)

In [ ]:
print("正在设置 Agent Identity 用户池...")
veidentity_config = setup_veidentity_user_pool()

print(veidentity_config)
print("Agent Identity 设置完成 ✓")

### 准备要部署到 AgentKit Runtime 的 agent

#### 使用火山方舟模型的 VeADK Agent

让我们基于[AgentKit Runtime 教程](https://volcengine.github.io/agentkit-sdk-python/content/4.runtime/1.runtime_quickstart.html)中创建的Agent开始，使用 Agent Identity 作为身份提供商配置入站身份验证。

In [ ]:
!mkdir -p test-agents

In [ ]:
%%writefile test-agents/agent.py

from veadk import Runner, Agent
from agentkit.apps import AgentkitSimpleApp
from veadk.tools.demo_tools import get_city_weather
app = AgentkitSimpleApp()
agent = Agent(model_name="doubao-seed-1-6-250615", tools=[get_city_weather])
runner = Runner(agent=agent)

@app.entrypoint
async def run(payload: dict, headers: dict) -> str:
    prompt = payload["prompt"]
    if prompt is None:
        return "prompt is None"

    user_id = headers.get("user_id", "testuser")
    session_id = headers.get("session_id", "testsession")

    response = await runner.run(
        messages=prompt,
        user_id=user_id,
        session_id=session_id
    )
    return response

@app.ping
def ping() -> str:
    return "pong!"

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=8000)


### 将 Agent 部署到 AgentKit Runtime

AgentKit Platform 的 `CreateRuntime` 操作支持全面的配置选项，让您可以指定容器镜像、环境变量和加密设置。您还可以配置协议设置（HTTP、MCP）和授权机制，以控制客户端与 Agent 的通信方式。

注意： 操作最佳实践是将代码打包为容器，结合火山引擎镜像仓库产品，通过 CI/CD 流水线和基础设施即代码（IaC）将镜像推送到容器镜像仓库。

在本教程中，我们将使用 AgentKit Python SDK 轻松打包您的代码并将其部署到 AgentKit Runtime 上。

#### 配置 AgentKit Runtime 部署

接下来，我们将使用 AgentKit SDK 来配置 AgentKit Runtime 部署，包括EntryPoint、用于访问火山方舟模型的API密钥以及运行的Python版本。

在配置步骤中，将根据您的应用程序代码自动生成 agentkit.yaml 配置文件，它用于在本地保存AgentKit Runtime的配置。

**重要** - 代码中通过`runtime_jwt_discovery_url`和`runtime_jwt_allowed_clients`指定了Inboud Auth 入站身份验证的配置，即：discoveryUrl 和allowedClients。

**说明** - 在本教程中使用 cloud 部署模式，即在远程构建镜像并自动部署到AgentKit Runtime上

In [ ]:
import os
from pathlib import Path
from agentkit.toolkit.sdk.config import AgentConfig

current_dir = os.getcwd()
# 如果当前目录不是 test-agents，则切换到该目录
if not current_dir.endswith("test-agents"):
    os.chdir(os.path.join(current_dir, "test-agents"))
agent_file = Path("agentkit.yaml")
if not agent_file.exists():
    agent_file.write_text("")
    print(f"Created empty file: {agent_file}")
else:
    print(f"File already exists, skipping: {agent_file}")

# 指定配置文件路径
config = AgentConfig("agentkit.yaml") \
    .set("common.agent_name", "veidentity_inbound_agent") \
    .set("common.entry_point", "agent.py") \
    .set("common.language", "Python") \
    .set("common.language_version", "3.12") \
    .set("common.launch_type", "cloud") \
    .set("launch_types.cloud.region", "cn-beijing") \
    .set("launch_types.cloud.runtime_id", "") \
    .set("launch_types.cloud.runtime_name", "") \
    .set("launch_types.cloud.runtime_role_name", "") \
    .set("launch_types.cloud.runtime_envs.MODEL_AGENT_API_KEY", MODEL_API_KEY) \
    .set("launch_types.cloud.runtime_auth_type", "custom_jwt") \
    .set("launch_types.cloud.runtime_jwt_discovery_url", veidentity_config.get("discovery_url")) \
    .set("launch_types.cloud.runtime_jwt_allowed_clients", [veidentity_config.get("client_id")]) \
    .save()

print("Configuration created and saved!")

#### 检查 AgentKit 配置

In [ ]:
!cat agentkit.yaml

#### 启动 AgentKit Runtime

现在我们有了配置文件，让我们将 Agent 启动到 AgentKit Runtime。这将会：

🔨 构建阶段 (Build Phase)
- 生成 Dockerfile - 根据项目配置自动生成容器化配置
- 构建 Docker 镜像 - 将你的 Agent 代码打包成容器镜像
- 准备镜像仓库 - 创建火山引擎 Container Registry (CR) 实例和命名空间
- 推送镜像 - 将构建好的镜像上传到云端镜像仓库

🚀 部署阶段 (Deploy Phase)
- 创建 IAM 角色 - 为 Runtime 生成必要的权限角色
- 创建 Runtime 实例 - 在火山引擎上创建新的 AgentKit Runtime
- 配置认证 - 设置 API Key 或 JWT 认证方式
- 等待就绪 - 监控 Runtime 状态直到达到 Ready 状态
- 获取访问端点 - 返回可用的公网访问地址

In [ ]:
launch_result = sdk.launch()
launch_result

### 测试 Agent 入站认证

#### 拒绝场景：在没有授权的情况下调用 AgentKit Runtime

最后，我们可以使用agent调用我们的 AgentKit Runtime。尝试运行以下单元格，您将看到一个错误，显示 **\"Authorization header with OAuth token is required for JWT authentication. Please provide headers={'Authorization': 'Bearer <your_oauth_token>'}\"。**

In [ ]:
invoke_response = sdk.invoke(
    payload={
        "prompt": "今天天气怎么样？",
    }
)
invoke_response

#### 正确访问场景：使用授权调用 AgentKit Runtime

让我们使用正确的授权令牌类型调用 Agent 。在我们的情况下，它将是 Agent Identity 访问令牌。从单元格 \"**配置 Agent Identity 用户池**\" 复制访问令牌

在当前示例中我们使用 `reauthenticate_user` 方法获取 Agent Identity 访问令牌。在生产环境中，您应该通过接入 Agent Identity SDK 或者调用 Agent Identity API 来获取访问令牌。

In [ ]:
bearer_token: str = reauthenticate_user(
    client_id=veidentity_config.get("client_id"),
    client_secret=veidentity_config.get("client_secret"),
    pool_id=veidentity_config.get("pool_id"),
    preferred_username=veidentity_config.get("preferred_username"),
)


invoke_response = sdk.invoke(
    payload={"prompt": "今天天气怎么样？"},
    headers={
        "user_id": "testuser",
        "session_id": "testsession",
        "Authorization": f"Bearer {bearer_token}",
    },
)
invoke_response

### 清理（可选）

- 现在让我们清理创建的 AgentKit Runtime

In [ ]:
destroy_result = sdk.destroy()
destroy_result

## 总结

您已成功完成了本教程！您学会了如何：

1. 配置 Agent Identity 用户池进行身份验证
2. 创建使用火山方舟模型的 VeADK  Agent 
3. 实现入站认证以保护 Agent 访问
4. 将agent部署到 AgentKit Runtime
5. 使用 JWT 令牌测试经过身份验证的agent调用

现在您可以扩展此示例以集成其他身份提供商或构建更复杂的认证工作流程。